# Análise comparativa de modelos

### Esse notebook destina-se a uma análise comparativa de diferentes abordagens para predição de sentimento em tweets. O objetivo final é analisar diferentes combinações de modelos, vetorizadores e redutores e seus respectivos hiperparâmetros para definir dentro todas as combinações possíveis aquela que tenha uma melhor desempenho geral. Para garantir isso será efetuada uma validação cruzada para cada combinação possível tanto de modelos, quanto de hiperparâmetros.

## Importando dependências

In [29]:
import pandas as pd
import numpy as np
import nltk
from joblib import dump
from sklearn.svm import SVC
from statistics import mode
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, RandomizedSearchCV, cross_validate
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import (
    CountVectorizer,
    TfidfVectorizer,
)

nltk.download("stopwords")
tweet_tokenizer = TweetTokenizer()


[nltk_data] Downloading package stopwords to /home/marvin-
[nltk_data]     linux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Será utilizado quatro dataframes oriundos da etapa de pre processamento, conforme o pipeline abaixo:

!["pipeline formatação de texto"](../notebooks/images/Pipeline_formata%C3%A7%C3%A3o_de_texto.png)


## Importando dados

In [30]:
df_stemming_sem_stopwords = pd.read_csv(
    "../data/processed/df_steamed_no_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
)

df_stemming_com_stopwords = pd.read_csv(
    "../data/processed/df_steamed_with_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
)

df_lematizado_sem_stopwords = pd.read_csv(
    "../data/processed/df_lemmetized_no_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
)

df_lematizado_com_stopwords = pd.read_csv(
    "../data/processed/df_lemmetized_with_stopwords.csv",
    usecols=["tweet_text", "sentiment"],
)


# Amostra de dados de cada dataset

In [31]:
df_lematizado_sem_stopwords.head(5)


,tweet_text,sentiment
38254,o mau ser aquele que não dever ser nomear ser ...,0
31927,gente isso ser real lt3,0
12522,sinceridade direto ser o chave usema d,1
57050,eu seguir q sigo de volta conta novo twitte...,0
39087,ter saudade de receber miminho,0


In [32]:
df_lematizado_com_stopwords.head(5)


,tweet_text,sentiment
38254,mau dever ser nomear ser eleger pq todo merda,0
31927,gente real lt3,0
12522,sinceridade direto chave usema d,1
57050,seguir q sigo voltar conta novo twitter bloque...,0
39087,saudade receber miminho,0


In [33]:
df_lematizado_sem_stopwords.head(5)


,tweet_text,sentiment
38254,o mau ser aquele que não dever ser nomear ser ...,0
31927,gente isso ser real lt3,0
12522,sinceridade direto ser o chave usema d,1
57050,eu seguir q sigo de volta conta novo twitte...,0
39087,ter saudade de receber miminho,0


In [34]:
df_stemming_com_stopwords.head(5)


,tweet_text,sentiment
38254,pi dev ser nome ser eleit pq tod merd,0
31927,gent real lt3,0
12522,sincer diret chav usem d,1
57050,seg q sig volt cont nov twitt bloqu antig,0
39087,saudad receb mim,0


# Verificação de tamanho

In [35]:
print(
    len(df_stemming_sem_stopwords),
    len(df_stemming_com_stopwords),
    len(df_lematizado_sem_stopwords),
    len(df_lematizado_com_stopwords),
)


3000 3000 3000 3000


# Separação dos textos do dataset e suas respectivas labels

### Positivo = 1
### Negativo  = 0

In [36]:
corpus_stemming_sem_stopwords = df_stemming_sem_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)

corpus_stemming_com_stopwords = df_stemming_com_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)
corpus_lematizado_sem_stopwords = df_lematizado_sem_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)
corpus_lematizado_com_stopwords = df_lematizado_com_stopwords.tweet_text.apply(
    lambda tweet_text: np.str_(tweet_text)
)

labels_stemming_sem_stopwords = df_stemming_sem_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()

labels_stemming_com_stopwords = df_stemming_com_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()

labels__lematizado_sem_stopwords = df_lematizado_sem_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()

labels_lematizado_com_stopwords = df_lematizado_com_stopwords.sentiment.replace(
    {"Positivo": 1, "Negativo": 0}
).to_list()


# Verificação do tamanho do vetor de tweets e labels

In [37]:
print(
    len(corpus_stemming_sem_stopwords),
    len(corpus_stemming_com_stopwords),
    len(corpus_lematizado_sem_stopwords),
    len(corpus_lematizado_com_stopwords),
    len(labels_stemming_sem_stopwords),
    len(labels_stemming_com_stopwords),
    len(labels__lematizado_sem_stopwords),
    len(labels_lematizado_com_stopwords),
)


3000 3000 3000 3000 3000 3000 3000 3000


### Inicialmente será definido um dicionário para cada uma  das etapas na formação de uma abordagem(pipeline), as etapas são:

!["pipeline approach"](../notebooks/images/pipeline_approach.png)

Cada modelo, vetorizador e redutores possui seu objeto e um conjunto de hiperparâmetros associados a ele. Para cada abordagem(vetorizador + redutor + modelo) será feita uma validação cruzada, para garantir a consistência das métricas. Além disso, para garantir uma competição justa, cada abordagem deve ser otimizada com os melhores hiperparâmetros possíveis, para que todas estejam em sua melhor versão. Em vista disso, também é necessário utilizar uma validação cruzada neles.

### A Definição dos componentes é feita abaixo


In [38]:
models = {
    "KNN": {
        "model_obj": KNeighborsClassifier(),
        "hyperparameters": {
            "n_neighbors": [7, 11, 21],
            "weights": ["uniform", "distance"],
        },
    },
    "SMV": {
        "model_obj": SVC(),
        "hyperparameters": {
            "kernel": ["linear", "poly", "rbf", "sigmoid"],
            "C": [0.1, 0.5, 1, 1.5, 2, 5, 10],
            "gamma": ["scale", "auto"],
        },
    },
    "GaussianNB": {
        "model_obj": GaussianNB(),
        "hyperparameters": {
            "var_smoothing": [
                1e-8,
                1e-6,
                1e-4,
                1e-2,
                1e-1,
                1e0,
            ]
        },
    },
}

vectorizers = {
    "TfidfVectorizer": {
        "vectorizer_obj": TfidfVectorizer(),
        "hyperparameters": {
            "max_features": [250, 500, 1000, 2000],
            "analyzer": ["word", "char"],
            "tokenizer": [tweet_tokenizer.tokenize, None],
        },
    },
    "CountVectorizer": {
        "vectorizer_obj": CountVectorizer(),
        "hyperparameters": {
            "max_features": [250, 500, 1000, 2000],
            "analyzer": ["word", "char"],
            "tokenizer": [tweet_tokenizer.tokenize, None],
        },
    },
}

reducer = {
    "TrucatedSVD": {
        "reducer_obj": TruncatedSVD(),
        "hyperparameters": {
            "n_components": [
                25,
                50,
                60,
            ]
        },
    },
}


corpus = {
    "corpus_stemming_sem_stopwords": {
        "corpus_data": corpus_stemming_sem_stopwords,
        "corpus_labels": labels_stemming_sem_stopwords,
    },
    "corpus_stemming_com_stopwords": {
        "corpus_data": corpus_stemming_com_stopwords,
        "corpus_labels": labels_stemming_com_stopwords,
    },
    "corpus_lematizado_sem_stopwords": {
        "corpus_data": corpus_lematizado_sem_stopwords,
        "corpus_labels": labels__lematizado_sem_stopwords,
    },
    "corpus_lematizado_com_stopwords": {
        "corpus_data": corpus_lematizado_com_stopwords,
        "corpus_labels": labels_lematizado_com_stopwords,
    },
}


# Abaixo para cada elemento utilizado foi feita uma descrição simplificado do seu funcionamento

# Modelos selecionados:  

- ### KNN (k-nearest neighbors):
   O algoritmo KNN é um dos algoritmos clássicos de aprendizado de máquina, usualmente utilizado como algoritmo de classificação a ideia básica proposta é que pontos semelhantes se encontra próximos um dos outros. Por se tratar de um algoritmo baseado na comparação de dados já existentes, o KNN é considerado um algoritmo do tipo "preguiçoso" já que basicamente decora os pontos do dataset, ou seja, o conhecimento já está diretamente nos dados e não em uma função preditora. No problema em questão sendo uma classificação binaria(positivo ou negativo) a classe definida será a que tiver mais de 50% dos votos. 
  
- Hiperparâmetros:
    - n_neighbors:
      Número de vizinho próximos a ser analisado. A quantidade de pontos é geralmente definida como um número impar para evitar empates na classificação de um novo dado, após isso a classe com maior número de instâncias será a selecionada.
      
    - weights:
      Define se a métrica utilizada será apenas a quantidade, ou se a distância dos pontos terá um peso.

- ### SVC (Support Vector Classification).
  O SVM funciona tentando criar uma hiperplano que separe linearmente os dados em classes diferentes, por exemplo, caso de uma plano 2d é simplesmente uma linha. O critério inicial para  isso é uma hiperplano é que ele consiga separar perfeitamente todos os dados, no caso de haver mais de um hiperplano que faça essa separação é definido como melhor aquele que maximiza a distância das instâncias de cada classe mais próxima. No caso dos dados não sejam linearmente separáveis a priore o SVM consegue aumentar quantidade de dimensões, tornando as classes separáveis dessa forma. No caso da análise de textos, montamos um vetor que represente aquele texto de alguma forma com n-dimensões para montar o hiperplano.

  - Hiperparâmetros:
    - kernel:
    Kernel utilizado para o aumento da dimensionalidade do modelo. Usualmente para aplicações de NLP o linear costuma ser o melhor
      
    - C:
    Parâmetro de regularização, "afrouxa" o critério de separação para ser possível separar mais facilmente os dados.

- ### Gaussian Naive Bayes
  O algoritmo Gausian Naive bayses consiste em fazer uma inferência baseado em várias curvas gaussianas adquiridas através das características do dataset de treino, onde cada uma delas é utilizada como uma parte para definir a probabilidade um dado ser de uma classe específica. No caso de um problema de NLP cada palavra possui sua curva gaussiana, associada com a probabilidade dela ser de uma classe ou outra. Em uma classificação binaria(positiva ou negativa), por exemplo, pode-se partir da pergunta: "Esse texto é positivo?" o algoritmo irá calcular a contagem de cada palavra presente no texto e repassar para cada curva gaussiana respectiva, no final irá tirar um score, a mesma coia será feita para a pergunta: "Esse texto é negativo?", calculando um novo score. Para a pergunta que obtiver o maior score será definida como a classe daquele novo input.
  - Hiperparâmetros:
    - var_smoothing:
    Porção utilizada da maior variância, influencia diretamente na geração da curva.

# Vetorizadores selecionados:  

- ### CountVectorizer
  Essa abordagem faz a contagem das palavras presente para cada uma das instâncias, no caso dessa aplicação tweets, as possibilidades são definidas baseadas no conjunto de todas as palavras possíveis de todos os tweets, o corpus. No final é gerado um vetor com a contagem de palavras presentes em cada tweet.

- Hiperparâmetros:
  - max_features:
    Define a quantidade máxima de palavras que será mantido a contagem, no caso o algoritmo sempre priorizará as palavras que mais aparecem, pois, elas têm um maior peso para a definição da classe.
  - analyzer:
    Define se o algoritmo irá analisar palavra ou letras para a contagem.
  - tokenizer:
    Define o critério usado para separar as palavras no texto para serem contadas, dependendo da origem do texto pode melhorar muito a análise.
    
    
- ### TfidfVectorizer
  Essa abordagem faz a contagem das palavras por instância(tweets) assim como a CountVectorizer, porém além disso calcula a frequência que essa palavra apareceu baseado em todas as instâncias. Ou seja, uma palavra que aparece muito em um determinado tweet, mas muito pouco nos demais, terá um peso muito maior para a definição da classe daquele tweet. Do contrário, uma palavra que aparece em abundância,  em um tweet, mas é muito comum em todos os outros terá um peso menor.
  
- Hiperparâmetros:
  - max_features:
    Define a quantidade máxima de palavras que será mantido a contagem, no caso o algoritmo sempre priorizará as palavras que mais aparecem, pois, elas têm um maior peso para a definição da classe.
  - analyzer:
    Define se o algoritmo irá analisar palavra ou letras para a contagem.
  - tokenizer:
    Define o critério usado para separar as palavras no texto para serem contadas, dependendo da origem do texto pode melhorar muito a análise
  

# Redutores selecionado:

- ### TruncatedSVD
  Geralmente modelos que trabalham com NPL não lidam bem com vetores com uma grande quantidade de zeros seguidos, devido a numerosa quantidade de palavras possíveis dentro do corpus, as instâncias(tweets) não possuirão a maioria das palavras possíveis no corpus, gerando o problema citado acima. Para contornar isso é necessário reduzir para uma dimensão menor esses dados  que já foram filtrados anteriormente na contagem sendo os mais relevantes. Aplicando o redutor de dimensionalidade SVD, esse vetor espaçado com zeros será reduzido.

- Hiperparâmetros:
  - n_components: Define para quantas features o vetor será reduzido.
  

# Treinamento
A seguir é feito o treinamento do modelo propriamente dito. Cada etapa será combinada com as demais, formando todas as possibilidades possíveis do conjunto de dados carregado anteriormente. Para cada uma dessas combinações é carregado um conjunto de hiperpârametros associados a elas. Devido à abundância de dados será utilizado apenas uma validação cruzada dos hiperparâmetro apresentados, sendo aplicado uma separação  holdout nos dados de texto, garantindo uma maior confiabilidade na escolha.

In [39]:
n_splits_cv = 1
n_splits_gs = 5

all_scores = {}

split_cv = ShuffleSplit(n_splits=n_splits_cv, test_size=0.2, random_state=42)

for corpus_name, corpus_data in corpus.items():

    for model_name, model_data in models.items():

        model_params = {
            f"model__{key}": value
            for key, value in model_data["hyperparameters"].items()
        }

        for vectorizer_name, vectorizer_data in vectorizers.items():

            vectorize_params = {
                f"vectorizer__{key}": value
                for key, value in vectorizer_data["hyperparameters"].items()
            }

            for reducer_name, reducer_data in reducer.items():

                reducer_params = {
                    f"reducer__{key}": value
                    for key, value in reducer_data["hyperparameters"].items()
                }

                param_distributions = {
                    **model_params,
                    **vectorize_params,
                    **reducer_params,
                }

                pipeline = Pipeline(
                    steps=[
                        ("vectorizer", vectorizer_data["vectorizer_obj"]),
                        ("reducer", reducer_data["reducer_obj"]),
                        ("scaler", StandardScaler()),
                        ("model", model_data["model_obj"]),
                    ]
                )

                approach_name = (
                    f"{corpus_name}__{model_name}__{vectorizer_name}__{reducer_name}"
                )

                print(f"Fiting best model to \n{approach_name}", end="\n\n")

                tuned_pipeline = RandomizedSearchCV(
                    pipeline,
                    param_distributions,
                    scoring="f1",
                    cv=n_splits_gs,
                    random_state=42,
                )

                scores = cross_validate(
                    tuned_pipeline,
                    corpus_data["corpus_data"],
                    corpus_data["corpus_labels"],
                    cv=split_cv,
                    scoring=["accuracy", "f1", "recall"],
                    ramdom_state=42,
                )

                all_scores.update(
                    {
                        approach_name: {
                            "scores": scores,
                        }
                    }
                )


Fiting best model to 
corpus_stemming_sem_stopwords__KNN__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__KNN__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__SMV__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__SMV__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__GaussianNB__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_sem_stopwords__GaussianNB__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__KNN__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__KNN__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__SMV__TfidfVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__SMV__CountVectorizer__TrucatedSVD

Fiting best model to 
corpus_stemming_com_stopwords__GaussianNB__TfidfVectorizer__TrucatedSVD

Fiting best

# Criação de um dataframe com todos os modelos criados

In [40]:
(
    approach_names,
    fit_times,
    scores_times,
    accuracy_means,
    f1_scores_mean,
    recall_scores_mean,
) = ([], [], [], [], [], [])


for approach_name, score in all_scores.items():
    approach_names.append(approach_name)
    fit_times.append(score["scores"]["fit_time"].mean())
    scores_times.append(score["scores"]["score_time"].mean())
    accuracy_means.append(score["scores"]["test_accuracy"].mean())
    f1_scores_mean.append(score["scores"]["test_f1"].mean())
    recall_scores_mean.append(score["scores"]["test_recall"].mean())

test_data = data = {
    "approach_name": approach_names,
    "fit_time": fit_times,
    "score_time": scores_times,
    "accuracy": accuracy_means,
    "f1": f1_scores_mean,
    "recall": recall_scores_mean,
}


test_data_df = pd.DataFrame(test_data)


test_data_df.style.background_gradient()


,approach_name,fit_time,score_time,accuracy,f1,recall
0,corpus_stemming_sem_stopwords__KNN__TfidfVectorizer__TrucatedSVD,11.665613,0.074983,0.630000,0.673529,0.715625
1,corpus_stemming_sem_stopwords__KNN__CountVectorizer__TrucatedSVD,12.766189,0.062117,0.596667,0.592593,0.550000
2,corpus_stemming_sem_stopwords__SMV__TfidfVectorizer__TrucatedSVD,17.323074,0.083924,0.681667,0.737997,0.840625
3,corpus_stemming_sem_stopwords__SMV__CountVectorizer__TrucatedSVD,19.049584,0.075728,0.683333,0.696486,0.681250
4,corpus_stemming_sem_stopwords__GaussianNB__TfidfVectorizer__TrucatedSVD,10.183861,0.029265,0.638333,0.703956,0.806250
5,corpus_stemming_sem_stopwords__GaussianNB__CountVectorizer__TrucatedSVD,10.272264,0.019836,0.565000,0.483168,0.381250
6,corpus_stemming_com_stopwords__KNN__TfidfVectorizer__TrucatedSVD,10.681467,0.067788,0.633333,0.671642,0.703125
7,corpus_stemming_com_stopwords__KNN__CountVectorizer__TrucatedSVD,10.198186,0.122363,0.645000,0.625659,0.556250
8,corpus_stemming_com_stopwords__SMV__TfidfVectorizer__TrucatedSVD,20.893663,0.060937,0.643333,0.719160,0.856250
9,corpus_stemming_com_stopwords__SMV__CountVectorizer__TrucatedSVD,33.679341,0.081970,0.666667,0.694190,0.709375


# Definição do melhor modelo, será utilizado a moda entre as métricas de f1, recall e accuracy fit_time e score_time, sendo essas duas ultimas menor melhor

In [41]:
approach_names = test_data_df.approach_name


def get_best_model(x):
    if x.name.endswith("time"):
        return approach_names[np.argmin(x.values)]

    return approach_names[np.argmax(x.values)]


In [42]:
best_approach_name = mode(test_data_df.apply(get_best_model, axis=0).to_list()).split(
    "__"
)
names = ["corpus", "model", "vectorizer", "reducer"]

best_approach_dict = {name: value for name, value in zip(names, best_approach_name)}

print(f"Best approach is \n{best_approach_dict}")


Best model is 
{'corpus': 'corpus_stemming_sem_stopwords', 'model': 'SMV', 'vectorizer': 'TfidfVectorizer', 'reducer': 'TrucatedSVD'}


In [43]:
best_reducer_hyperparameters = reducer[best_approach_dict["reducer"]]["hyperparameters"]
best_vectorizer_hyperparameters = vectorizers[best_approach_dict["vectorizer"]][
    "hyperparameters"
]
best_model_hyperparameters = models[best_approach_dict["model"]]["hyperparameters"]


reducer_params = {
    f"reducer__{key}": value for key, value in best_reducer_hyperparameters.items()
}
vectorize_params = {
    f"vectorizer__{key}": value
    for key, value in best_vectorizer_hyperparameters.items()
}
model_params = {
    f"model__{key}": value for key, value in best_model_hyperparameters.items()
}


param_distributions = {
    **model_params,
    **vectorize_params,
    **reducer_params,
}


best_obj_vectorizer = vectorizers[best_approach_dict["vectorizer"]]["vectorizer_obj"]

best_obj_reducer = reducer[best_approach_dict["reducer"]]["reducer_obj"]

best_obj_model = models[best_approach_dict["model"]]["model_obj"]

corpus_final = corpus[best_approach_dict["corpus"]]["corpus_data"]
labels_final = corpus[best_approach_dict["corpus"]]["corpus_labels"]


# Após a definição da melhor abordagem é necessário fazer novamente a validação cruzada dos hiperparâmetros, mas agora com todos os dados disponíveis, para garantir uma generalização maior do modelo

In [44]:
cv_n_splits_final = 5

pipeline = Pipeline(
    steps=[
        (
            "vectorizer",
            best_obj_vectorizer,
        ),
        (
            "reducer",
            best_obj_reducer,
        ),
        ("scaler", StandardScaler()),
        ("model", best_obj_model),
    ]
)
tuned_pipeline = RandomizedSearchCV(
    pipeline,
    param_distributions,
    scoring="f1",
    cv=cv_n_splits_final,
    random_state=42,
)


scores = cross_validate(
    tuned_pipeline,
    corpus_final,
    labels_final,
    cv=split_cv,
    scoring=["accuracy", "f1", "recall"],
)


# Finalizado tudo o melhor modelo é ajustado e persistido

In [45]:
tuned_pipeline.fit(corpus_final, labels_final)

dump(tuned_pipeline.best_estimator_, "../models/tuned_pipeline.joblib")


['../models/tuned_pipeline.joblib']

In [46]:
y_hat = tuned_pipeline.best_estimator_.predict(corpus_final)

confusion_matrix(labels_final, y_hat)


array([[1321,  222],
       [  27, 1430]])